# **<span style="color:blue">크롤링한 데이터를 가져오고 이미지를 생성</span>**
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

data_kor = pd.read_csv("/content/drive/MyDrive/플밍2프로젝트/kor_scraping.csv") # 한국 뉴스를 크롤링하여 저장한 csv를 불러옴
data_kor.head()

,Unnamed: 0,제목,분야,일자,언론사,댓글수,관련뉴스개수,댓글수_W,관련뉴스개수_W,W,Select,요약,번역,이미지
0,0,"국민의힘, 국회서 ‘공매도 개선’ 민당정 협의회",정치,2023-11-16,경향신문,11,81,0.181818,1.0,0.672727,1,금융위 공매도 제도개선 방향 설명금감원 불법 공매도 조사현황 보고\n이날 협의회...,"At the Council, the Financial Services Commiss...",kor_image_0
1,1,"삼성전자·LG전자, 'CES 2024' 혁신상 휩쓸어",경제,2023-11-16,연합뉴스,1,53,1.000000,1.0,1.000000,1,삼성전자는 영상디스플레이 14개 모바일 5개 생활가전 8개 등 총 28개의 혁신...,Samsung received a total of 28 innovation awar...,kor_image_1
2,2,[속보] 尹 대통령 장모 ‘잔고증명 위조 혐의’ 징역 1년 확정,사회,2023-11-16,문화일보,315,95,1.000000,1.0,1.000000,1,윤석열 대통령의 장모 최은순 씨가 지난 7월 21일 의정부지법에서 열린 항소심 선고...,President Yoon Seok -yeol's mother -in -law Ch...,kor_image_2
3,3,자율주행차에 신호정보 0.1초 내 전달…경찰·현대기아차 협업,생활_문화,2023-11-16,연합뉴스,0,20,0.000000,1.0,0.600000,1,협약에 따라 세 기관은 경찰청이 제공하는 실시간 교통 신호정보 를 바탕으로 실...,"Under the agreement, the three organizations i...",kor_image_3
4,4,윤 대통령 ‘지스타’서 이례적 영상 축사 “지스타 게임산업 중추적 역할”,IT_과학,2023-11-16,부산일보,14,56,1.000000,1.0,1.000000,1,16일 벡스코서 개막식 열려평소 게임산업 관심 반영 후문\n16일 오전 부산 해운대...,"On the 16th, the opening ceremony of BEXCO wil...",kor_image_4


In [ ]:
data_eng = pd.read_csv("/content/drive/MyDrive/플밍2프로젝트/eng_scraping.csv") # 해외 뉴스를 크롤링하여 저장한 csv를 불러옴
data_eng.head()

,제목,분야,요약,번역_kor,이미지
0,'Did we die?' A week in the life of a journali...,정치,I missed the days we all were living in our bu...,나는 우리 모두가 이스라엘에 의해 파괴 된 가자 시티의 우리 건물에 살고있는 날을 ...,eng_image_0
1,Black Friday shopping deals have started. Here...,경제,In the last several years retailers have star...,지난 몇 년 동안 소매 업체는 Black Friday 또는 Cyber Monday ...,eng_image_1
2,Joe Biden falls behind Donald Trump with young...,선거_사회,WASHINGTON President Joe Biden is trailing f...,조 비덴 (Joe Biden) 워싱턴 대통령은 일요일에 발표 된 NBC 뉴스 여론 ...,eng_image_2
3,"Horoscopes Today, November 20, 2023",생활,Is it in the stars Free Daily and Monthly Hor...,매일 별이없는 별과 매월 별자리가 매일 운세를 읽습니다.Sun 화성과 명왕성이 만날...,eng_image_3
4,Has Elon Musk gone too far? Outrage grows over...,기술,It is unacceptable to repeat the hideous lie b...,홀로 코스트 백악관 대변인 앤드류 베이츠 (Andrew Bates)가 성명서에서 밝혔다.,eng_image_4


In [ ]:
pip install diffusers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:00


In [ ]:
pip install invisible_watermark transformers accelerate safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 10.5 MB/s eta 0:00:00


In [ ]:
pip install Pillow

In [ ]:
from diffusers import DiffusionPipeline # 이미지 생성 AI를 가져올 수 있는 모듈
import torch # diffusers를 사용할 때 기반이 되는 모듈로, 혹시 몰라서 임포트함
from PIL import Image # 이미지 파일을 조작할 수 있는 라이브러리

# 모델 가져오기
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# 한국 뉴스 기사 이미지 생성
import os # 이미지를 생성하고 나서 저장하기 위해 사용하는 라이브러리

output_path = "/content/drive/MyDrive/플밍2프로젝트/"

prompts = data_kor["번역"] # 번역할 텍스트를 가져옴

for i, prompt in enumerate(prompts): # 인덱스와 텍스트를 가져옴
    result_image = pipe(prompt=prompt).images[0] # 모델로 이미지를 생성하고 저장
    filename = f"kor_image_{i}.png"
    file_path = os.path.join(output_path, filename)
    result_image.save(file_path)

# 해외 뉴스 기사 이미지 생성
prompts = data_eng["요약"] # 번역할 텍스트를 가져옴

for i, prompt in enumerate(prompts): # 인덱스와 텍스트를 가져옴
    result_image = pipe(prompt=prompt).images[0] # 모델로 이미지를 생성하고 저장
    filename = f"eng_image_{i}.png"
    file_path = os.path.join(output_path, filename)
    result_image.save(file_path)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

이미지 생성 소요 시간: 710.5602719783783
